# AI Agent Chain: Chain of Responsibility Pattern in Action

This notebook demonstrates how to use the **Chain of Responsibility** pattern to build intelligent AI agents that can handle different types of requests, similar to how LangChain and LangGraph create agent workflows.

## Pattern Overview
The Chain of Responsibility pattern allows a request to pass through a chain of handlers until one can process it. In AI applications, this is perfect for:
- Multi-step reasoning
- Specialized agent workflows  
- Fallback mechanisms
- Request routing

In [12]:
# Import required modules
import sys
sys.path.append('.')

from utils.client import AIClient, MockAIClient
from abc import ABC, abstractmethod
from typing import Optional, Dict, Any, List

# First, let's define the Chain of Responsibility pattern base class
class Handler(ABC):
    """
    Base Handler class for Chain of Responsibility pattern
    
    This pattern allows passing requests along a chain of handlers. 
    Upon receiving a request, each handler decides either to process 
    the request or to pass it to the next handler in the chain.
    """
    
    def __init__(self, successor: Optional["Handler"] = None):
        """
        Initialize handler with optional successor
        Args:
            successor: Next handler in the chain
        """
        self.successor = successor
    
    @abstractmethod
    def handle(self, request):
        """
        Handle the request or pass to successor
        This method should be implemented by concrete handlers
        """
        pass

print("✅ Chain of Responsibility pattern base class defined!")
print("📖 This pattern helps us create flexible agent workflows where:")
print("   - Each agent specializes in specific tasks")  
print("   - Requests flow through the chain until handled")
print("   - Easy to add/remove/reorder agents")
print("   - Clean separation of concerns")

✅ Chain of Responsibility pattern base class defined!
📖 This pattern helps us create flexible agent workflows where:
   - Each agent specializes in specific tasks
   - Requests flow through the chain until handled
   - Easy to add/remove/reorder agents
   - Clean separation of concerns


## Understanding Chain of Responsibility Pattern

Before diving into AI agents, let's understand what makes this pattern so powerful:

### 🔗 **The Chain Concept**
Think of it like a **customer support system**:
1. **Level 1 Support** → handles basic questions
2. **Level 2 Technical** → handles complex technical issues  
3. **Level 3 Specialist** → handles very specific problems
4. **Manager** → handles escalated cases

Each level tries to help, and if they can't, they pass it up the chain!

### **Why Perfect for AI Agents?**
- **Specialization**: Each AI agent can focus on what it does best
- **Flexibility**: Easy to add new types of agents or reorder them
- **Fallback**: If no agent can handle something, we have a backup plan
- **Scalability**: Chain can grow without breaking existing agents

Let's build this step by step!

In [13]:
class AIAgentHandler(Handler):
    """Base AI Agent Handler"""
    
    def __init__(self, successor: Optional["AIAgentHandler"] = None, use_mock: bool = True):
        super().__init__(successor)
        self.client = MockAIClient() if use_mock else AIClient()
        self.agent_name = self.__class__.__name__
    
    def handle(self, request: Dict[str, Any]) -> Optional[str]:
        """Handle request and return response or pass to next handler"""
        if self.can_handle(request):
            return self.process_request(request)
        elif self.successor:
            return self.successor.handle(request)
        else:
            return "No handler available for this request."
    
    @abstractmethod
    def can_handle(self, request: Dict[str, Any]) -> bool:
        """Check if this handler can process the request"""
        pass
    
    @abstractmethod
    def process_request(self, request: Dict[str, Any]) -> str:
        """Process the request and return response"""
        pass

## Specialized Agent Implementations

In [14]:
class CodeAgent(AIAgentHandler):
    """Handles programming and code-related queries"""
    
    def can_handle(self, request: Dict[str, Any]) -> bool:
        query = request.get('query', '').lower()
        code_keywords = ['code', 'program', 'function', 'algorithm', 'debug', 'python', 'javascript']
        return any(keyword in query for keyword in code_keywords)
    
    def process_request(self, request: Dict[str, Any]) -> str:
        prompt = f"As a coding expert, help with: {request['query']}"
        response = self.client.generate_text(prompt)
        return f"🤖 {self.agent_name}: {response}"


class MathAgent(AIAgentHandler):
    """Handles mathematical queries and calculations"""
    
    def can_handle(self, request: Dict[str, Any]) -> bool:
        query = request.get('query', '').lower()
        math_keywords = ['calculate', 'math', 'equation', 'solve', 'formula', 'statistics']
        return any(keyword in query for keyword in math_keywords)
    
    def process_request(self, request: Dict[str, Any]) -> str:
        prompt = f"As a math expert, solve: {request['query']}"
        response = self.client.generate_text(prompt)
        return f"📊 {self.agent_name}: {response}"


class GeneralAgent(AIAgentHandler):
    """Handles general queries that don't fit other categories"""
    
    def can_handle(self, request: Dict[str, Any]) -> bool:
        return True  # Always handles as fallback
    
    def process_request(self, request: Dict[str, Any]) -> str:
        prompt = f"Please help with: {request['query']}"
        response = self.client.generate_text(prompt)
        return f"🧠 {self.agent_name}: {response}"

## Setting Up the Agent Chain

Now let's create our agent chain, similar to LangChain's agent workflows:

In [16]:
# Create agent chain
general_agent = GeneralAgent(use_mock=True)
math_agent = MathAgent(successor=general_agent, use_mock=True)
code_agent = CodeAgent(successor=math_agent, use_mock=True)

print("🔗 AI Agent Chain Created:")
print("CodeAgent → MathAgent → GeneralAgent")

INFO:utils.client:🧪 MockAIClient initialized for testing (provider: mock)
INFO:utils.client:🧪 MockAIClient initialized for testing (provider: mock)
INFO:utils.client:🧪 MockAIClient initialized for testing (provider: mock)


🔗 AI Agent Chain Created:
CodeAgent → MathAgent → GeneralAgent


## Testing the Agent Chain

Let's test our agent chain with different types of queries:

In [17]:
# Test queries
test_queries = [
    {"query": "Write a Python function to calculate fibonacci numbers", "type": "code"},
    {"query": "Solve the equation: 2x + 5 = 15", "type": "math"},
    {"query": "What is the capital of France?", "type": "general"},
    {"query": "Debug this Python code that has syntax errors", "type": "code"},
    {"query": "Calculate the area of a circle with radius 5", "type": "math"}
]

print("🚀 Testing Agent Chain:\n")

for i, query in enumerate(test_queries, 1):
    print(f"Query {i} ({query['type']}): {query['query']}")
    response = code_agent.handle(query)
    print(f"Response: {response}\n")
    print("-" * 80)

🚀 Testing Agent Chain:

Query 1 (code): Write a Python function to calculate fibonacci numbers
Response: 🤖 CodeAgent: Mock coding response: Here's a simple Python function for your request about 'As a coding expert, help with:...'

--------------------------------------------------------------------------------
Query 2 (math): Solve the equation: 2x + 5 = 15
Response: 📊 MathAgent: Mock math response: The solution to your mathematical query 'As a math expert, solve: Solve...' is calculated as follows.

--------------------------------------------------------------------------------
Query 3 (general): What is the capital of France?
Response: 🧠 GeneralAgent: Mock general response: This is a helpful response to your query about 'Please help with: What is the ...'

--------------------------------------------------------------------------------
Query 4 (code): Debug this Python code that has syntax errors
Response: 🤖 CodeAgent: Mock coding response: Here's a simple Python function for your 

## Advanced: Stateful Agent Chain

Let's create a more sophisticated version that maintains state across requests, similar to LangGraph's state management:

In [18]:
class StatefulAgentHandler(AIAgentHandler):
    """Agent with state management"""
    
    def __init__(self, successor: Optional["StatefulAgentHandler"] = None, use_mock: bool = True):
        super().__init__(successor, use_mock)
        self.state = {"processed_count": 0, "history": []}
    
    def handle(self, request: Dict[str, Any]) -> Optional[str]:
        # Update state
        if self.can_handle(request):
            self.state["processed_count"] += 1
            self.state["history"].append(request["query"])
            
            response = self.process_request(request)
            return f"{response} [Processed: {self.state['processed_count']} requests]"
        elif self.successor:
            return self.successor.handle(request)
        return "No handler available."


class StatefulCodeAgent(StatefulAgentHandler):
    def can_handle(self, request: Dict[str, Any]) -> bool:
        query = request.get('query', '').lower()
        return 'code' in query or 'program' in query or 'python' in query
    
    def process_request(self, request: Dict[str, Any]) -> str:
        return f"💻 StatefulCodeAgent: Handling code request..."


class StatefulGeneralAgent(StatefulAgentHandler):
    def can_handle(self, request: Dict[str, Any]) -> bool:
        return True
    
    def process_request(self, request: Dict[str, Any]) -> str:
        return f"🎯 StatefulGeneralAgent: Handling general request..."

In [19]:
# Create stateful agent chain
stateful_general = StatefulGeneralAgent(use_mock=True)
stateful_code = StatefulCodeAgent(successor=stateful_general, use_mock=True)

print("🔄 Testing Stateful Agent Chain:\n")

stateful_queries = [
    {"query": "Write Python code for sorting"},
    {"query": "What's the weather like?"},
    {"query": "Create a function in Python"},
    {"query": "Tell me a joke"}
]

for query in stateful_queries:
    response = stateful_code.handle(query)
    print(f"Query: {query['query']}")
    print(f"Response: {response}\n")

print(f"📈 Code Agent State: {stateful_code.state}")
print(f"📈 General Agent State: {stateful_general.state}")

INFO:utils.client:🧪 MockAIClient initialized for testing (provider: mock)
INFO:utils.client:🧪 MockAIClient initialized for testing (provider: mock)


🔄 Testing Stateful Agent Chain:

Query: Write Python code for sorting
Response: 💻 StatefulCodeAgent: Handling code request... [Processed: 1 requests]

Query: What's the weather like?
Response: 🎯 StatefulGeneralAgent: Handling general request... [Processed: 1 requests]

Query: Create a function in Python
Response: 💻 StatefulCodeAgent: Handling code request... [Processed: 2 requests]

Query: Tell me a joke
Response: 🎯 StatefulGeneralAgent: Handling general request... [Processed: 2 requests]

📈 Code Agent State: {'processed_count': 2, 'history': ['Write Python code for sorting', 'Create a function in Python']}
📈 General Agent State: {'processed_count': 2, 'history': ["What's the weather like?", 'Tell me a joke']}


## Real-world Applications

This pattern is particularly useful for:

### 1. **LangChain-style Agent Workflows**
- Router agents that direct queries to specialized tools
- Multi-step reasoning chains
- Tool selection and execution

### 2. **LangGraph-inspired State Management**
- Stateful conversations across multiple turns
- Context preservation between agent interactions
- Complex workflow orchestration

### 3. **Production AI Systems**
- Content moderation pipelines
- Multi-modal processing (text → image → audio)
- Error handling and fallback mechanisms

## Advanced 2: LangGraph SubGraph Concept

Now let's explore how to create **SubGraphs** - treating entire agent chains as building blocks for larger systems. This is the core concept behind LangGraph's architecture.

### 🔗 **SubGraph = Chain as a Single Unit**

Think of SubGraph as **"chains within chains"**:
- A SubGraph encapsulates multiple agents into one logical unit
- The MainGraph orchestrates multiple SubGraphs
- Each SubGraph can be reused, tested, and modified independently


In [20]:
class SubGraphHandler:
    """
    SubGraph: Treat entire agent chain as a single processing unit
    This mimics LangGraph's subgraph concept where complex workflows
    can be encapsulated and reused as building blocks
    """
    
    def __init__(self, name: str, internal_agents: List[str], use_mock: bool = True):
        self.name = name
        self.internal_agents = internal_agents
        self.client = MockAIClient(f"subgraph-{name}") if use_mock else AIClient("gemini")
        self.processed_requests = []
    
    def process(self, request: str) -> Dict[str, Any]:
        """Process request through internal agent chain"""
        print(f"    🔸 SubGraph '{self.name}' processing...")
        
        # Simulate chain processing through multiple agents
        for i, agent in enumerate(self.internal_agents, 1):
            print(f"      → Step {i}: {agent} analyzing...")
        
        # Generate response using the subgraph's specialized prompt
        specialized_prompt = f"As a {self.name} expert team, help with: {request}"
        response = self.client.generate_text(specialized_prompt)
        
        # Track processed requests
        self.processed_requests.append(request)
        
        return {
            "subgraph": self.name,
            "agents_used": self.internal_agents,
            "steps_count": len(self.internal_agents),
            "response": response,
            "total_processed": len(self.processed_requests)
        }

class MainGraphOrchestrator:
    """
    Main Graph that orchestrates multiple SubGraphs
    Similar to LangGraph's main workflow with embedded subgraphs
    """
    
    def __init__(self, use_mock: bool = True):
        print("🏗️ Building LangGraph-style SubGraph Architecture...")
        
        # Create specialized subgraphs - each is a complete workflow
        self.subgraphs = {
            "technical": SubGraphHandler(
                name="TechnicalSupport", 
                internal_agents=["CodeReviewer", "DebugAnalyzer", "ArchitecturalAdvisor", "TestGenerator"],
                use_mock=use_mock
            ),
            "analytical": SubGraphHandler(
                name="DataAnalytics", 
                internal_agents=["DataValidator", "StatisticalAnalyzer", "VisualizationGenerator", "InsightExtractor"],
                use_mock=use_mock
            ),
            "creative": SubGraphHandler(
                name="ContentCreation", 
                internal_agents=["IdeaGenerator", "ContentWriter", "StyleOptimizer", "QualityReviewer"],
                use_mock=use_mock
            ),
            "general": SubGraphHandler(
                name="GeneralAssistance", 
                internal_agents=["QueryClassifier", "InformationRetriever", "ResponseSynthesizer"],
                use_mock=use_mock
            )
        }
        
        print(f"✅ Created {len(self.subgraphs)} specialized subgraphs")
    
    def route_request(self, request: str) -> Dict[str, Any]:
        """Intelligent routing to appropriate subgraph (like LangGraph's conditional edges)"""
        request_lower = request.lower()
        
        # Smart routing logic
        if any(word in request_lower for word in ['code', 'bug', 'debug', 'program', 'algorithm', 'technical']):
            chosen_subgraph = "technical"
        elif any(word in request_lower for word in ['analyze', 'data', 'calculate', 'statistics', 'chart', 'math']):
            chosen_subgraph = "analytical"
        elif any(word in request_lower for word in ['write', 'create', 'story', 'article', 'content', 'blog']):
            chosen_subgraph = "creative"
        else:
            chosen_subgraph = "general"
        
        print(f"  🎯 MainGraph routing to '{chosen_subgraph}' subgraph")
        result = self.subgraphs[chosen_subgraph].process(request)
        
        return {
            "main_graph_decision": chosen_subgraph,
            "subgraph_result": result,
            "total_steps": result["steps_count"] + 1  # +1 for routing step
        }
    
    def get_system_stats(self) -> Dict[str, Any]:
        """Get statistics about the entire system"""
        stats = {}
        total_processed = 0
        
        for name, subgraph in self.subgraphs.items():
            count = len(subgraph.processed_requests)
            stats[name] = count
            total_processed += count
        
        return {
            "subgraph_usage": stats,
            "total_requests_processed": total_processed,
            "most_used_subgraph": max(stats.keys(), key=lambda k: stats[k]) if total_processed > 0 else "none"
        }

# Create the MainGraph system
print("🚀 Creating LangGraph-style System with SubGraphs:")
main_system = MainGraphOrchestrator(use_mock=True)

INFO:utils.client:🧪 MockAIClient initialized for testing (provider: subgraph-TechnicalSupport)
INFO:utils.client:🧪 MockAIClient initialized for testing (provider: subgraph-DataAnalytics)
INFO:utils.client:🧪 MockAIClient initialized for testing (provider: subgraph-ContentCreation)
INFO:utils.client:🧪 MockAIClient initialized for testing (provider: subgraph-GeneralAssistance)


🚀 Creating LangGraph-style System with SubGraphs:
🏗️ Building LangGraph-style SubGraph Architecture...
✅ Created 4 specialized subgraphs


In [21]:
# Demonstrate SubGraph Flexibility - Creating Different MainGraph Configurations
print("🔧 SubGraph Flexibility Demo - Different System Configurations:\n")

# Configuration 1: Customer Support System
class CustomerSupportSystem(MainGraphOrchestrator):
    def __init__(self, use_mock: bool = True):
        print("🏢 Building Customer Support System...")
        self.subgraphs = {
            "technical_support": SubGraphHandler(
                name="TechnicalSupport", 
                internal_agents=["IssueClassifier", "TroubleshootingAgent", "EscalationHandler"],
                use_mock=use_mock
            ),
            "billing_support": SubGraphHandler(
                name="BillingSupport", 
                internal_agents=["AccountVerifier", "PaymentProcessor", "RefundHandler"],
                use_mock=use_mock
            ),
            "general_inquiry": SubGraphHandler(
                name="GeneralInquiry", 
                internal_agents=["FAQMatcher", "InfoProvider", "FollowUpScheduler"],
                use_mock=use_mock
            )
        }

# Configuration 2: Content Production System  
class ContentProductionSystem(MainGraphOrchestrator):
    def __init__(self, use_mock: bool = True):
        print("📝 Building Content Production System...")
        self.subgraphs = {
            "research": SubGraphHandler(
                name="ResearchTeam", 
                internal_agents=["TopicAnalyzer", "SourceGatherer", "FactChecker", "OutlineCreator"],
                use_mock=use_mock
            ),
            "writing": SubGraphHandler(
                name="WritingTeam", 
                internal_agents=["ContentWriter", "StyleEditor", "SEOOptimizer"],
                use_mock=use_mock
            ),
            "production": SubGraphHandler(
                name="ProductionTeam", 
                internal_agents=["MediaCreator", "QualityReviewer", "PublishingCoordinator"],
                use_mock=use_mock
            )
        }

# Show different system configurations
print("System 1: Customer Support")
support_system = CustomerSupportSystem(use_mock=True)
print(f"   SubGraphs: {list(support_system.subgraphs.keys())}")

print("\\nSystem 2: Content Production") 
content_system = ContentProductionSystem(use_mock=True)
print(f"   SubGraphs: {list(content_system.subgraphs.keys())}")

print("\\n🎯 Key Insight: Same SubGraph Pattern, Different Business Logic!")
print("   - Reusable architecture patterns")
print("   - Domain-specific implementations") 
print("   - Easy to create new system types")

print("\\n🚀 This demonstrates the power of the SubGraph pattern:")
print("   ✅ One pattern, infinite applications")
print("   ✅ Business logic separated from architecture") 
print("   ✅ Rapid prototyping of complex systems")
print("   ✅ Perfect foundation for LangGraph-style AI workflows!")

INFO:utils.client:🧪 MockAIClient initialized for testing (provider: subgraph-TechnicalSupport)
INFO:utils.client:🧪 MockAIClient initialized for testing (provider: subgraph-BillingSupport)
INFO:utils.client:🧪 MockAIClient initialized for testing (provider: subgraph-GeneralInquiry)
INFO:utils.client:🧪 MockAIClient initialized for testing (provider: subgraph-ResearchTeam)
INFO:utils.client:🧪 MockAIClient initialized for testing (provider: subgraph-WritingTeam)
INFO:utils.client:🧪 MockAIClient initialized for testing (provider: subgraph-ProductionTeam)


🔧 SubGraph Flexibility Demo - Different System Configurations:

System 1: Customer Support
🏢 Building Customer Support System...
   SubGraphs: ['technical_support', 'billing_support', 'general_inquiry']
\nSystem 2: Content Production
📝 Building Content Production System...
   SubGraphs: ['research', 'writing', 'production']
\n🎯 Key Insight: Same SubGraph Pattern, Different Business Logic!
   - Reusable architecture patterns
   - Domain-specific implementations
   - Easy to create new system types
\n🚀 This demonstrates the power of the SubGraph pattern:
   ✅ One pattern, infinite applications
   ✅ Business logic separated from architecture
   ✅ Rapid prototyping of complex systems
   ✅ Perfect foundation for LangGraph-style AI workflows!


## Testing the SubGraph System

Let's see how our LangGraph-style system handles different types of requests by routing them to appropriate subgraphs:

In [22]:
# Test the SubGraph system with diverse requests
subgraph_test_cases = [
    "Debug my Python sorting algorithm that's throwing index errors",
    "Analyze sales data trends for Q4 and create visualizations", 
    "Write a compelling blog post about AI ethics",
    "What's the best way to learn machine learning?",
    "Fix this React component that won't render properly",
    "Calculate correlation between user engagement and revenue"
]

print("🧪 Testing LangGraph SubGraph System:\n")

results = []
for i, test_case in enumerate(subgraph_test_cases, 1):
    print(f"📝 Request {i}: {test_case}")
    
    result = main_system.route_request(test_case)
    results.append(result)
    
    subgraph_info = result["subgraph_result"]
    print(f"✅ Processed by: {subgraph_info['subgraph']} ({subgraph_info['steps_count']} steps)")
    print(f"📄 Response: {subgraph_info['response'][:100]}...")
    print(f"📊 This subgraph has handled: {subgraph_info['total_processed']} requests")
    print("-" * 60)

# Show system statistics
print(f"\n📈 System Statistics:")
stats = main_system.get_system_stats()
print(f"Total requests processed: {stats['total_requests_processed']}")
print(f"Most used subgraph: {stats['most_used_subgraph']}")

print(f"\n📊 SubGraph Usage Breakdown:")
for subgraph_name, count in stats['subgraph_usage'].items():
    percentage = (count / stats['total_requests_processed'] * 100) if stats['total_requests_processed'] > 0 else 0
    print(f"  {subgraph_name}: {count} requests ({percentage:.1f}%)")

🧪 Testing LangGraph SubGraph System:

📝 Request 1: Debug my Python sorting algorithm that's throwing index errors
  🎯 MainGraph routing to 'technical' subgraph
    🔸 SubGraph 'TechnicalSupport' processing...
      → Step 1: CodeReviewer analyzing...
      → Step 2: DebugAnalyzer analyzing...
      → Step 3: ArchitecturalAdvisor analyzing...
      → Step 4: TestGenerator analyzing...
✅ Processed by: TechnicalSupport (4 steps)
📄 Response: Mock coding response: Here's a simple Python function for your request about 'As a TechnicalSupport ...
📊 This subgraph has handled: 1 requests
------------------------------------------------------------
📝 Request 2: Analyze sales data trends for Q4 and create visualizations
  🎯 MainGraph routing to 'analytical' subgraph
    🔸 SubGraph 'DataAnalytics' processing...
      → Step 1: DataValidator analyzing...
      → Step 2: StatisticalAnalyzer analyzing...
      → Step 3: VisualizationGenerator analyzing...
      → Step 4: InsightExtractor analyzing...



## SubGraph Architecture Benefits

### 🏗️ **LangGraph SubGraph Pattern Advantages:**

#### **1. Hierarchical Composition**
```
MainGraph (Level 0)
├── TechnicalSupport SubGraph (Level 1)
│   ├── CodeReviewer → DebugAnalyzer → ArchitecturalAdvisor → TestGenerator
├── DataAnalytics SubGraph (Level 1)  
│   ├── DataValidator → StatisticalAnalyzer → VisualizationGenerator → InsightExtractor
└── ContentCreation SubGraph (Level 1)
    ├── IdeaGenerator → ContentWriter → StyleOptimizer → QualityReviewer
```

#### **2. Real-World Parallel**
Think of SubGraphs like **specialized departments** in a company:
- **IT Department** (TechnicalSupport SubGraph) → handles all tech issues
- **Data Science Team** (DataAnalytics SubGraph) → handles all analytical work  
- **Marketing Team** (ContentCreation SubGraph) → handles all creative content
- **Customer Service** (GeneralAssistance SubGraph) → handles everything else

#### **3. Key Benefits:**
- **🔹 Encapsulation**: Each subgraph is self-contained with its own workflow
- **🔹 Reusability**: Subgraphs can be used in multiple main graphs
- **🔹 Testability**: Each subgraph can be tested independently
- **🔹 Maintainability**: Changes to one subgraph don't affect others
- **🔹 Scalability**: Add new subgraphs without modifying existing code
- **🔹 Performance**: Parallel processing of different subgraphs

### 🔄 **Pattern Hierarchy We've Built:**

1. **Chain of Responsibility** (Level 1) → Simple agent chains
2. **SubGraph Pattern** (Level 2) → Multiple chains as building blocks  
3. **MainGraph Orchestration** (Level 3) → System-level coordination

This is exactly how **LangGraph** structures complex AI workflows! 🎯